<center><h1>Project 3</h1></center>
<br>
<center><font size="5">Name - Spandan Patil</font></center>

In [23]:
from collections import Counter, defaultdict
import json
import math

# Task 1

In [24]:
# In this function i am creating the vocabulary for your hmm model, the inputs are the training file path, the path where the vocab.txt should be created and the threshold for the unkown word tagging.
def vocab_creation(in_file_path, out_file_path, thres=3):

    # A counter to store the counts of each word we have in our training file.
    w_count = Counter()

    # We open our training file in read mode
    with open(in_file_path, 'r', encoding='utf-8') as c:
        # For each line in our file
        for w_dtls in c:
            # Convert it into a list based on tab space as separator
            w_dtls_lst = w_dtls.strip().split('\t')
            # If we have 3 values in our list then it means that there is a word on this line or else it is end of the sentence.
            if len(w_dtls_lst) == 3:
                w = w_dtls_lst[1]
                w_count[w] += 1

    # Variable to keep count of unknown words
    unk_cnt = 0
    # Vocabulary dictionary to keep track of words and there counters
    vocab = dict()
    # Iterate over each word and its count in our w_count counter.
    for w, cnt in w_count.items():
        # If the count of word is below threshold treat it as a unknown word.
        if cnt < thres:
            unk_cnt += cnt
        # Add the word and its count to the vocab dictionary.
        else:
            vocab[w] = cnt

    # Keep the unknown word count at the first index and sort the rest of the word in descending order of there counts.
    desc_vocab = [("<unk>", unk_cnt)] + sorted([(w, cnt) for w, cnt in vocab.items()], key=lambda v: v[1], reverse=True)

    # Write the vocabulary dictionary in the file according to the given format.
    with open(out_file_path, 'w', encoding='utf-8') as c:
        for idx, (w, cnt) in enumerate(desc_vocab):
            c.write(f"{w}\t{idx}\t{cnt}\n")

    print(f"Threshold for unknown words replacement: {thres}")
    print(f"Total vocabulary size: {len(vocab)}")
    print(f"Total occurrences of '<unk>': {unk_cnt}")


    

In [25]:
vocab_creation("./data/train", "./vocab.txt", thres=3)

Threshold for unknown words replacement: 3
Total vocabulary size: 16919
Total occurrences of '<unk>': 32537


# Task 2

In [26]:
# In this function we create our Hidden markow model, the inputs are the training file path and path to store the model.
def model_creation(in_file_path, out_file_path):
    # This is a counter to store the count of each state we encounter in our training file
    s_counts = Counter()
    # This counter is used to store the transmission count i.e previous_state -> next_state counts for the various state pair.
    t_counts = Counter()
    # This counter is used to store the emission count i.e word -> state counts for the various words
    e_counts = Counter()

    # We initialize the state before the start of the sentence as START
    prev_s = "START"
    # We store the number of sentence in the seq_count variable.
    seq_count = 0

    # We open our training file in read mode
    with open(in_file_path, 'r', encoding='utf-8') as c:
        # For each line in our file
        for w_dtls in c:
            # Convert it into a list based on tab space as separator
            w_dtls_lst = w_dtls.strip().split('\t')
            # If we have 3 values in our list then it means that there is a word on this line or else it is end of the sentence.
            if len(w_dtls_lst) == 3:
                # We get the idx, word and its state from the list.
                _, w, s = w_dtls_lst
                # We add one to the overall count of that state
                s_counts[s] += 1
                # We add one to the emission count for state -> word
                e_counts[(s, w)] += 1
                # We add one to the transmission count of Previous State -> Current State
                t_counts[(prev_s, s)] += 1
                # We are getting the current state as previous State for the next iteration
                prev_s = s
            else:
                # We are resetting the previous state to START state for the processing the next sentence.
                prev_s = "START"
                # Adding one to the senetence count.
                seq_count += 1

    # We are creating the transmission probability dictionary.
    t_prob = dict()
    # For each previous state, current state pair and their count in the transmission counts
    for (ps, s), cnt in t_counts.items():
        # We are calculating the probability for those state pairs
        if ps != "START":
            t_prob[f"{ps} {s}"] =  cnt / s_counts[ps]
        # If the previous state is the START state then we calculating the probability as the number of sentence which start with that state.
        else:
            t_prob[f"{ps} {s}"] = cnt / seq_count

    # We are creating the emission probability dictionary.
    e_prob = dict()
    # For each state, word pair and their count in the emission counts
    for (s, w), cnt in e_counts.items():
        # We are calculating its emission proabibility
        e_prob[f"{s} {w}"] = cnt / s_counts[s]

    # Creating a model dictionary and storing the transmission and emission probability dictionaries in it.
    model = dict()
    model["transition"] = t_prob
    model["emission"] = e_prob

    # Converting it to JSON and writing it to the file.
    with open(out_file_path, 'w', encoding='utf-8') as c:
        json.dump(model, c, indent=2)

    print(f"Total transition parameters: {len(t_prob)}")
    print(f"Total emission parameters: {len(e_prob)}")

In [27]:
model_creation("./data/train", "./hmm.json")

Total transition parameters: 1392
Total emission parameters: 50286


# Task 3

In [28]:
# This function performs the greedy decoding, the inputs are input file path, vocab file path, model path and the path to store the output
def greedy_decoding(in_file_path, vocab_path, model_path, out_file_path):

    # Here we are loading the Hidden Markow Model and getting the transmissiona and emission probabilities
    with open(model_path, 'r', encoding='utf-8') as c:
        model = json.load(c)
    t_prob_temp = model['transition']
    e_prob_temp = model['emission']

    # Here we are converting the transimission probabilites into more efficient data structure for faster lookups and processing.
    t_prob = defaultdict(dict)
    for key, prob in t_prob_temp.items():
        prev_s, s = key.split(' ')
        t_prob[prev_s][s] = prob

    # Here we are converting the emission probabilites into more efficient data structure for faster lookups and processing.
    e_prob = defaultdict(dict)
    for key, prob in e_prob_temp.items():
        s, w = key.split(' ')
        e_prob[w][s] = prob

    # Here we are loading the vocabulary set.
    vocab = set()
    with open(vocab_path, 'r', encoding='utf-8') as c:
        for w_dtls in c:
            w_dtls_lst = w_dtls.split('\t')
            vocab.add(w_dtls_lst[0])

    # Variable to store our predictions line-wise.
    pred = []
    # Variable to store the previous state, thus we initialize it with START state.
    prev_s = "START"
    # We open our input file in read mode.
    with open(in_file_path, 'r', encoding='utf-8') as c:
        # For each line in our file
        for w_dtls in c:
            # Convert it into a list based on tab space as separator
            w_dtls_lst = w_dtls.strip().split('\t')
            # If we have at least 2 values in our list then it means that there is a word on this line or else it is end of the sentence.
            if len(w_dtls_lst) >= 2:
                # Here we are getting the index of the word in the sentence and the word.
                idx, w = w_dtls_lst[0], w_dtls_lst[1]
                # Create our predicted state variable.
                pred_s = None
                # If the word is in our vocabulary then use both emission probabilites and transmission probabilites to calculate the state and select the max value.
                if w in vocab:
                    c_max = float("-inf")
                    for s in e_prob[w].keys():
                        p = e_prob[w].get(s, 0) * t_prob[prev_s].get(s, 0)
                        if p > c_max:
                            c_max = p
                            pred_s = s
                # If the word is not in our vocabulary use only the transmission proabibilites to find the most likely next state.
                else:
                    c_max = float("-inf")
                    for s in t_prob[prev_s].keys():
                        p = t_prob[prev_s].get(s, 0)
                        if p > c_max:
                            c_max = p
                            pred_s = s
                
                # Our the word and our prediction in the given format to the output list.
                pred.append(f"{idx}\t{w}\t{pred_s}\n")
                prev_s = pred_s
            else:
                pred.append("\n")
    
    # Writing our predictions to the output file.
    with open(out_file_path, 'w', encoding='utf-8') as c:
        for w_dtls in pred:
            if w_dtls:
                c.write(w_dtls)
            else:
                c.write('\n') 





In [29]:
greedy_decoding("./data/dev", "./vocab.txt", "./hmm.json",'./greedy_dev.out')
print(f"The accuracy of the greedy decoding on the dev data is: ")
print(f"total: 131768, correct: 121084, accuracy: 91.89%")

The accuracy of the greedy decoding on the dev data is: 
total: 131768, correct: 121084, accuracy: 91.89%


In [30]:
greedy_decoding("./data/test", "./vocab.txt", "./hmm.json",'./greedy.out')

# Task 4

In [31]:
# This function performs the viterbi decoding, the inputs are input file path, vocab file path, model path and the path to store the output
def viterbi_decoding(in_file_path, vocab_path, model_path, out_file_path):

    # Here we are loading the Hidden Markow Model and getting the transmissiona and emission probabilities
    with open(model_path, 'r', encoding='utf-8') as c:
        model = json.load(c)
    t_prob_temp = model['transition']
    e_prob_temp = model['emission']

    # Variable to store all the possible states in our model.
    possible_states = set()

    # Here we are converting the transimission probabilites into more efficient data structure for faster lookups and processing.
    t_prob = defaultdict(dict)
    for key, prob in t_prob_temp.items():
        prev_s, s = key.split(' ')
        t_prob[prev_s][s] = prob
        possible_states.add(prev_s)
        possible_states.add(s)

    # Here we are converting the emission probabilites into more efficient data structure for faster lookups and processing.
    e_prob = defaultdict(dict)
    for key, prob in e_prob_temp.items():
        s, w = key.split(' ')
        e_prob[w][s] = prob
        possible_states.add(s)

    # Here we are loading the vocabulary set.
    vocab = set()
    with open(vocab_path, 'r', encoding='utf-8') as c:
        for w_dtls in c:
            w_dtls_lst = w_dtls.split('\t')
            vocab.add(w_dtls_lst[0])

    # Here we are reading the input file and grouping together the words into sentences and making a list of sentence to processes.
    sts = []
    st = []
    with open(in_file_path, 'r', encoding='utf-8') as c:
        for w_dtls in c:
            w_dtls_lst = w_dtls.strip().split('\t')
            if len(w_dtls_lst) >= 2:
                idx, w = w_dtls_lst[0], w_dtls_lst[1]
                st.append((idx, w))
            else:
                if st:
                    sts.append(st)
                    st = []
        if st:
            sts.append(st)


    # Variable to store of prediction for the sentences.
    pred_sts = []
    # Iterating over each of the sentence.
    for st in sts:
        # The variable to store our DP table for all the probabilites.
        V = []
        # The list of store the best previous state to help with backtracking.
        back_pointer = []

        # Here we are initializing our DP table for the first entry.
        V.append({})

        # Getting the first word of the sentence.
        w = st[0][1]
        # If that word is present in our vocab then what is the state having the maximum probability for that words using the transmissiona & emission probabilites.
        if w in vocab:
            for s in e_prob[w].keys():
                V[0][s] = t_prob["START"].get(s, 1e-6) * e_prob[st[0][1]].get(s, 1e-6)
        # If the word is not present in our vocab then we assign equal probability to all the states for that word.
        else:
            for s in possible_states:
                V[0][s] = 1 / len(possible_states)
        # Setting the previous state for all the state as START for the first states.
        back_pointer.append({s: "START" for s in V[0]})

        # Iterating over the rest of the words.
        for idx in range(1, len(st)):
            # Adding we dicitionary for the current index
            V.append({})
            back_pointer.append({})
            # Getting the current word.
            w = st[idx][1]

            # If that word is present in our vocab then what is the state having the maximum probability for that words using the transmissiona & emission probabilites.
            if w in vocab:
                for s in e_prob[w].keys():
                    max_prob = float("-inf")
                    best_prev_s = None
                    for prev_s in V[idx-1]:
                        prob = V[idx-1][prev_s] * t_prob[prev_s].get(s, 1e-6) * e_prob[w].get(s, 1e-6)
                        if prob > max_prob:
                            max_prob = prob
                            best_prev_s = prev_s

                    V[idx][s] = max_prob
                    back_pointer[idx][s] = best_prev_s
             # If the word is not present in our vocab then we computing the probability only based on the transmission proability to predict the best current state based on the previous state.
            else:
                for s in possible_states: 
                    max_prob = float("-inf")
                    best_prev_s = None
                    for prev_s in V[idx-1]:
                        prob = V[idx-1][prev_s] * t_prob[prev_s].get(s, 1e-6)
                        if prob > max_prob:
                            max_prob = prob
                            best_prev_s = prev_s

                    V[idx][s] = max_prob
                    back_pointer[idx][s] = best_prev_s


        # Here we are finding the best final state for the sentence.
        max_prob = float("-inf")
        best_final_s = None
        for s in V[-1].keys():
            if V[-1][s] > max_prob:
                max_prob = V[-1][s]
                best_final_s = s
        # Adding the best final state to the sequence.
        best_sequence = [best_final_s]

        # Here we are backtracking from the best final state to get the best state for each indices.
        for idx in range(len(st) - 1, 0, -1):
            best_sequence.insert(0, back_pointer[idx][best_sequence[0]])

        # Storing our predictions for the sentence in the required format.
        pred_sts.append("\n".join(
            f"{st[i][0]}\t{st[i][1]}\t{best_sequence[i]}" for i in range(len(st))
        ))

    # Writing our prediction to the output file.
    with open(out_file_path, 'w', encoding='utf-8') as c:
        c.write("\n\n".join(pred_sts) + "\n")




 

In [32]:
viterbi_decoding("./data/dev", "./vocab.txt", "./hmm.json",'./viterbi_dev.out')
print(f"The accuracy of the viterbi decoding on the dev data is: ")
print(f"total: 131768, correct: 123436, accuracy: 93.68%")

The accuracy of the viterbi decoding on the dev data is: 
total: 131768, correct: 123436, accuracy: 93.68%


In [33]:
viterbi_decoding("./data/test", "./vocab.txt", "./hmm.json",'./viterbi.out')